# **MACHINE LEARNING NANODEGREE**
***
## **PROJETO FINAL**
***

## **Análise Preditiva de Cota para Exercício Parlamentar (CEAP)**

## ** PARTE 3 - Predição de Dados**
***

## ** DEPUTADO DOMINGOS NETO**

***

## ** INTRODUÇÃO **

Realizaremos uma análise preditiva dos gastos para o ano de 2018 utilizando-se de algoritmo de aprendizado supervisionado.

É importante ressaltar que um deputado, salvo reeleição, terá registro de apenas quatros anos de despezas da CEAP. O que acaba sendo relativamente poucos dados os três primeiros anos para previsão dos gastos do último ano do mandato. Esta característica do dataset pode causar overfitting e ter impactos negativos com outliers.

Para contornar este problema devemos buscar utilizar algoritmos de menor complexidade, com menor número de parâmetros.
Tendo em vista estes desafios iremos utilizar dois tipos de lagoritmo:
> 1. LINEAR REGRESSION
> 2. LASSO REGRESSION
> 3. LARS LASSO REGRESSION
> 4. RIDGE REGRESSION
> 5. BAYESIAN RIDGE

Referência: https://goo.gl/gRYnWW

***

## **A. Importação de Biblioteca Python**

#### Serão utilizadas neste projeto as seguintes bibliotecas Python:

> 1. Pandas
> 2. Numpy
> 3. Scikit Learn
> 4. Seaborn


In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.metrics import r2_score
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns

# pre-processing
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import LabelEncoder

# Cross validation
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import cross_val_predict

# Model selection
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold

# Machine learning

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.grid_search import GridSearchCV

# Métrica
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.metrics import make_scorer

# definindo que a execução do código na linha de baixo.
%matplotlib inline

# Ajustando globalmente o impedimento de notação científica
pd.options.display.float_format = '{:20,.2f}'.format

# Variável para definir a amostragem para head() e Tail()
H = 1

/Users/tec/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


***

## **B. Preparação de Dataset**


### B.1 Importando o dataset
Será utilizado o dataset individual criado para o **Dep. Domingos Neto**:

> - df_dom_con.csv.


In [2]:
display(os.listdir('dataset'))

['.DS_Store',
 'Ano-2015.csv',
 'Ano-2016.csv',
 'Ano-2017.csv',
 'df_dom_con.csv',
 'df_fco_con.csv',
 'df_roc_con.csv',
 'df_trienio_limpo.csv']

In [3]:
# importando o dataset limpo.
df = pd.read_csv('dataset/df_dom_con.csv', sep = ';', encoding='latin1')

In [4]:
df.Tipo.unique()

array(['C&L', 'Consultorias', 'Correios', 'Divulgação', 'Hospedagem',
       'Passagens Aéreas', 'S/D', 'Telefonia', 'Escritório',
       'Locação de Carro'], dtype=object)

***
## C. DEPUTADO DOMINGOS NETO

## C.1 CRIANDO X e y

In [5]:
# Criação da variáveis X e y.
X = df
y = X.pop('Valor')

In [6]:
X.head(H)

,Tipo,Ano
0,C&L,2015


In [7]:
y.head(H)

0              15,222.61
Name: Valor, dtype: float64

In [8]:
X.shape

(24, 2)

## C.2 CRIANDO UM DATAFRAME PARA PREDIÇÃO DE 2018

Antes de aplicarmos get_dummies na feature Tipo, precisamos listar todo o conteúdo único contido para criarmos o dataframe de predição para 2018.

In [9]:
uniq = X.Tipo.unique()
#display(uniq)
uniq = pd.DataFrame(uniq, columns=['Tipo'])
#display(uniq)
df_2018 = pd.get_dummies(uniq, columns = ['Tipo'])
#display(df_tipo)
df_2018['Ano'] = 2018
#ano = df_2018.pop('Ano')
#df_2018['Ano'] = ano
df_2018.head(12)
#ano = df_2018.pop('Ano')
df_2018 = df_2018.drop('Ano', axis = 1)
df_2018.insert(0, 'Ano', 2018)
df_2018.head(20)

,Ano,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Telefonia
0,2018,1,0,0,0,0,0,0,0,0,0
1,2018,0,1,0,0,0,0,0,0,0,0
2,2018,0,0,1,0,0,0,0,0,0,0
3,2018,0,0,0,1,0,0,0,0,0,0
4,2018,0,0,0,0,0,1,0,0,0,0
5,2018,0,0,0,0,0,0,0,1,0,0
6,2018,0,0,0,0,0,0,0,0,1,0
7,2018,0,0,0,0,0,0,0,0,0,1
8,2018,0,0,0,0,1,0,0,0,0,0
9,2018,0,0,0,0,0,0,1,0,0,0


## C.3 TRATANDO FEATURE COM CLASSIFICAÇÃO
Tendo em vista que os algoritmos de machine learning da biblioteca Scikit Learn não tratam automáticamente com colunas com tipo de dados Objeto utilizaremos a função get_dummies do Pandas.
Esta função irá criar colunas adicionais para representar cada tipo de gasto existente e indicará através de 0 e 1 se ela está presente na linha ou não.
Maiores informações podem ser encontradas neste link: 
> - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

In [10]:
# Vamos utilizar get_dummies para tratar os dados não numéricos da feature 'Tipo'.
X = pd.get_dummies(X, columns = ['Tipo'])

In [11]:
X.head(H)

,Ano,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Telefonia
0,2015,1,0,0,0,0,0,0,0,0,0


In [12]:
X.shape

(24, 11)

## C.4 'LOGARITMIZAÇÃO'

In [13]:
# Aplicação de logaritmica
    
yl = np.log(y)
Xl = X

## C.5 DATA SPLIT

Como estaremos realizando o uso dos dados de 2015, 2016 e 2017 para realizar a predição dos gastos de 2018 de três deputados, iremos então separar os dados de 2015 e 2016 para serem utilizados como train e os dados de 2017 para test.
O splot de dados será realizado COM e SEM a aplicação de log em y.

In [14]:
# Test split percentual e random state.
per = 0.20
random = 7

# data split SEM LOG.
# Data split com 20% de dados para treinamento.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = per, random_state = random)

# data split COM LOG.
# Data split com 20% de dados para treinamento.
Xl_train, Xl_test, yl_train, yl_test = train_test_split(Xl, yl, test_size = per, random_state = random)

In [15]:
X_train.head(H)

,Ano,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Telefonia
21,2017,0,0,0,0,0,0,0,1,0,0


In [16]:
X_test.head(H)

,Ano,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Telefonia
1,2015,0,1,0,0,0,0,0,0,0,0


In [17]:
y_train.head(H)

21             134,878.66
Name: Valor, dtype: float64

In [18]:
yl_train.head(H)

21                  11.81
Name: Valor, dtype: float64

In [19]:
y_test.head(H)

1             402,000.00
Name: Valor, dtype: float64

In [20]:
yl_test.head(H)

1                  12.90
Name: Valor, dtype: float64

## C.6 PREDIÇÃO POR REGRESSÃO LINEAR

### CRIAÇÃO DE FUNÇÃO PARA CONSTRUÇÃO DE TABELA RESULTADO


In [21]:
# Criando tabela com o resultado da regressão
def tabela_resultado(modelo, df):
    pr = modelo.predict(df)
    pr = pd.DataFrame(data=pr)
    #display(pr)
    pr['Valor'] = pr
    #display(pr)
    pr = pr.pop('Valor')
    #display(pr)
    co = df_2018.columns
    co = pd.DataFrame(data=co)
    #display(co)
    co.drop(0, axis = 0, inplace = True)
    co['Tipo'] = co
    co = co.pop('Tipo')
    co = co.reset_index(drop=True)
    #display(co)
    results = pd.concat([co, pr], axis = 1)
    return results
    #display(results)

### CRIAÇÃO DE DATAFRAME PARA PREDIÇÃO PARA 2018

***
## D. LINEAR REGRESSION

### D.1 SEM APLICAÇÃO DE  LOG

In [22]:
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)
display('Score para train ', lr.score(X_train,y_train).round(4))
display('Score para test ', lr.score(X_test,y_test).round(4))


'Score para train '

0.8993

'Score para test '

0.8026

### D.2 PREDIÇÃO PARA 2018 SEM LOG

In [23]:
# Predição
# Data recover
resultado = tabela_resultado(lr, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"74,160.85"
1,Tipo_Consultorias,"370,114.64"
2,Tipo_Correios,"34,446.40"
3,Tipo_Divulgação,"93,746.19"
4,Tipo_Escritório,"68,935.82"
5,Tipo_Hospedagem,"133,444.86"
6,Tipo_Locação de Carro,"90,846.19"
7,Tipo_Passagens Aéreas,"51,048.07"
8,Tipo_S/D,"45,750.69"
9,Tipo_Telefonia,"106,943.75"


1069437.46

***
### D.3 COM APLICAÇÃO DE  LOG

In [24]:
lrl = linear_model.LinearRegression()
lrl.fit(Xl_train,yl_train)
display('Score para train ', lrl.score(Xl_train,yl_train).round(4))
display('Score para test ', lrl.score(Xl_test,yl_test).round(4))


'Score para train '

0.9183

'Score para test '

0.9432

### D.4 PREDIÇÃO PARA 2018 COM LOG

In [25]:
# Predição
#tabela_resultado(lrl, df_2018)
# Data recover
resultado = tabela_resultado(lrl, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"59,336.39"
1,Tipo_Consultorias,"585,752.51"
2,Tipo_Correios,562.71
3,Tipo_Divulgação,"100,516.80"
4,Tipo_Escritório,"2,338.14"
5,Tipo_Hospedagem,"168,727.77"
6,Tipo_Locação de Carro,"48,240.95"
7,Tipo_Passagens Aéreas,"10,166.93"
8,Tipo_S/D,587.37
9,Tipo_Telefonia,"18,239.57"


994469.15

***

***
## E. LASSO REGRESSION

***

### E.1 SEM APLICAÇÃO DE LOG

In [26]:
from sklearn import linear_model
la = linear_model.Lasso(alpha = 100)
la.fit(X_train,y_train)
display('Score para train ', la.score(X_train,y_train).round(4))
display('Score para test ', la.score(X_test,y_test).round(4))

'Score para train '

0.8992

'Score para test '

0.8216

### E.2 PREDIÇÃO PARA 2018  SEM LOG

In [27]:
resultado = tabela_resultado(la, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"74,409.05"
1,Tipo_Consultorias,"369,305.84"
2,Tipo_Correios,"35,540.27"
3,Tipo_Divulgação,"92,994.82"
4,Tipo_Escritório,"71,141.30"
5,Tipo_Hospedagem,"132,645.77"
6,Tipo_Locação de Carro,"90,419.17"
7,Tipo_Passagens Aéreas,"51,890.61"
8,Tipo_S/D,"47,850.51"
9,Tipo_Telefonia,"74,409.05"


1040606.4

***

### E.3 COM APLICAÇÃO DE LOG

In [28]:
from sklearn import linear_model
lal = linear_model.Lasso(alpha = 100)
lal.fit(Xl_train,yl_train)
display('Score para train ', lal.score(Xl_train,yl_train).round(4))
display('Score para test ', lal.score(Xl_test,yl_test).round(4))

'Score para train '

0.0

'Score para test '

-0.052

### E.4 PREDIÇÃO PARA 2018 COM LOG

In [29]:
resultado = tabela_resultado(lal, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"12,418.94"
1,Tipo_Consultorias,"12,418.94"
2,Tipo_Correios,"12,418.94"
3,Tipo_Divulgação,"12,418.94"
4,Tipo_Escritório,"12,418.94"
5,Tipo_Hospedagem,"12,418.94"
6,Tipo_Locação de Carro,"12,418.94"
7,Tipo_Passagens Aéreas,"12,418.94"
8,Tipo_S/D,"12,418.94"
9,Tipo_Telefonia,"12,418.94"


124189.42

***

***

## F. LARS LASSO

### F.1 SEM APLICAÇÃO DE LOG

In [30]:
from sklearn import linear_model
ll = linear_model.LassoLars(alpha=.1)
ll.fit(X_train,y_train)
display('Score para train ', ll.score(X_train,y_train).round(4))
display('Score para test ', ll.score(X_test,y_test).round(4))

'Score para train '

0.8993

'Score para test '

0.8255

### F.2 PREDIÇÃO PARA 2018 SEM LOG

In [31]:
resultado = tabela_resultado(ll, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"74,160.59"
1,Tipo_Consultorias,"370,112.83"
2,Tipo_Correios,"34,447.13"
3,Tipo_Divulgação,"93,744.20"
4,Tipo_Escritório,"68,936.60"
5,Tipo_Hospedagem,"133,443.05"
6,Tipo_Locação de Carro,"90,844.47"
7,Tipo_Passagens Aéreas,"51,048.36"
8,Tipo_S/D,"45,751.82"
9,Tipo_Telefonia,"74,160.59"


1036649.65

### F.3 COM APLICAÇÃO DE LOG

In [32]:
from sklearn import linear_model
lll = linear_model.LassoLars(alpha=.1)
lll.fit(Xl_train,yl_train)
display('Score para train ', lll.score(Xl_train,yl_train).round(4))
display('Score para test ', lll.score(Xl_test,yl_test).round(4))

'Score para train '

0.6775

'Score para test '

0.6799

### F.4 PREDIÇÃO PARA 2018 COM LOG

In [33]:
resultado = tabela_resultado(lll, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"18,150.42"
1,Tipo_Consultorias,"96,029.18"
2,Tipo_Correios,"1,171.65"
3,Tipo_Divulgação,"18,150.42"
4,Tipo_Escritório,"5,072.54"
5,Tipo_Hospedagem,"27,661.50"
6,Tipo_Locação de Carro,"18,150.42"
7,Tipo_Passagens Aéreas,"13,588.54"
8,Tipo_S/D,"1,823.34"
9,Tipo_Telefonia,"18,150.42"


217948.41

***

***

## G. RIDGE REGRESSION

### G.1 SEM APLICAÇÃO DE LOG

In [34]:
from sklearn import linear_model
ri = linear_model.Ridge (alpha = .5)
ri.fit(X_train,y_train)
display('Score para train ', ri.score(X_train,y_train).round(4))
display('Score para test ', ri.score(X_test,y_test).round(4))

'Score para train '

0.8659

'Score para test '

0.6562

### G.2 PREDIÇÃO PARA 2018 SEM LOG

In [35]:
resultado = tabela_resultado(ri, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"87,323.10"
1,Tipo_Consultorias,"324,324.03"
2,Tipo_Correios,"55,789.44"
3,Tipo_Divulgação,"104,904.94"
4,Tipo_Escritório,"93,011.08"
5,Tipo_Hospedagem,"134,988.21"
6,Tipo_Locação de Carro,"101,624.82"
7,Tipo_Passagens Aéreas,"67,512.15"
8,Tipo_S/D,"74,761.55"
9,Tipo_Telefonia,"116,026.59"


1160265.91

### G.3 COM APLICAÇÃO DE LOG

In [36]:
from sklearn import linear_model
ril = linear_model.Ridge (alpha = .5)
ril.fit(Xl_train,yl_train)
display('Score para train ', ril.score(Xl_train,yl_train).round(4))
display('Score para test ', ril.score(Xl_test,yl_test).round(4))

'Score para train '

0.8717

'Score para test '

0.8348

### G.4 PREDIÇÃO PARA 2018 COM LOG

In [37]:
resultado = tabela_resultado(ril, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"60,307.17"
1,Tipo_Consultorias,"342,476.01"
2,Tipo_Correios,"1,320.33"
3,Tipo_Divulgação,"86,484.95"
4,Tipo_Escritório,"6,025.75"
5,Tipo_Hospedagem,"126,534.24"
6,Tipo_Locação de Carro,"50,501.85"
7,Tipo_Passagens Aéreas,"13,294.90"
8,Tipo_S/D,"2,267.55"
9,Tipo_Telefonia,"24,095.30"


713308.06

***

***

## H. BAYESIAN RIDGE

### H.1 SEM APLICAÇÃO DE LOG

In [38]:
br = linear_model.BayesianRidge()
br.fit(X_train,y_train)
display('Score para train ', br.score(X_train,y_train).round(4))
display('Score para test ', br.score(X_test,y_test).round(4))

'Score para train '

0.0

'Score para test '

-0.1961

### H.2 PREDIÇÃO PARA 2018 SEM LOG

In [39]:
resultado = tabela_resultado(br, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"63,558.14"
1,Tipo_Consultorias,"63,558.14"
2,Tipo_Correios,"63,558.14"
3,Tipo_Divulgação,"63,558.14"
4,Tipo_Escritório,"63,558.14"
5,Tipo_Hospedagem,"63,558.14"
6,Tipo_Locação de Carro,"63,558.14"
7,Tipo_Passagens Aéreas,"63,558.14"
8,Tipo_S/D,"63,558.14"
9,Tipo_Telefonia,"63,558.14"


635581.37

***

### H.3 COM APLICAÇÃO DE LOG

In [40]:
brl = linear_model.BayesianRidge()
brl.fit(Xl_train,yl_train)
display('Score para train ', brl.score(Xl_train,yl_train).round(4))
display('Score para test ', brl.score(Xl_test,yl_test).round(4))

'Score para train '

0.911

'Score para test '

0.9087

### H.4 PREDIÇÃO PARA 2018 COM LOG

In [41]:
resultado = tabela_resultado(brl, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_C&L,"60,498.61"
1,Tipo_Consultorias,"480,268.64"
2,Tipo_Correios,779.55
3,Tipo_Divulgação,"95,856.83"
4,Tipo_Escritório,"3,509.82"
5,Tipo_Hospedagem,"151,971.71"
6,Tipo_Locação de Carro,"49,714.36"
7,Tipo_Passagens Aéreas,"11,354.50"
8,Tipo_S/D,"1,035.15"
9,Tipo_Telefonia,"20,629.50"


875618.68

Referências:
> 1. https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
> 2.  https://chrisalbon.com/machine_learning/preprocessing_structured_data/preprocessing_categorical_features/
> 3. https://www.packtpub.com/mapt/book/big_data_and_business_intelligence/9781787286382/2/ch02lvl1sec23/working-with-categorical-variables
> 4. http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
> 5. http://scikit-learn.org/stable/auto_examples/plot_cv_predict.html#sphx-glr-auto-examples-plot-cv-predict-py
> 6. http://fastml.com/how-to-use-pd-dot-get-dummies-with-the-test-set/
> 7. Scikit-learn Cookbook - Second Edition - Julian Avila, Trent Hauck - November 2017 - https://www.packtpub.com/mapt/book/all_books/9781787286382
> 8. scikit-learn : Machine Learning Simplified - Raúl Garreta, Guillermo Moncecchi, Trent Hauck, Gavin Hackeling - November 2017 - https://www.packtpub.com/mapt/book/all_books/9781788833479
> 9. Mastering Machine Learning with scikit-learn - Second Edition - Gavin Hackeling - July 2017 - https://www.packtpub.com/mapt/book/all_books/9781788299879
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 